<a href="https://colab.research.google.com/github/AlexSan1910/Tratamento_De_Dados_PySpark/blob/main/PySpark_S3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
a = 10
b = 15
c = a ** b
c

1000000000000000

In [ ]:
c - 1000000000

999999000000000

In [ ]:
# criando ambiente spark:

# ! = comumente utilizado para gerar comandos no sistema operacional e não um comando python

# Instalando o Pyspark

!pip install Pyspark

In [ ]:
# Importando biblioteca pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("SENAI").getOrCreate()

In [ ]:
# Criando um DataFrame normalmente atraves de listas:

dados = [
    ("joao", 25, "Analista", 3800),
    ("maria", 38, "Analista", 5500),
    ("marcos", 22, "Estagiario", 1000),
    ("fabio", 35, "Assistente", 2500),
    ("luis", 40, "Gerente", 11700),
    ("ana", 39, "Gerente", 15900)
]

In [ ]:
# Cabeçalho (Header):

colunas = ["nome", "idade", "cargo", "salario"]

In [ ]:
# Criando o DataFrame Spark:

df = spark.createDataFrame(dados, colunas)

In [ ]:
# Exibindo informações do DataFrame
df.show()

+------+-----+----------+-------+
|  nome|idade|     cargo|salario|
+------+-----+----------+-------+
|  joao|   25|  Analista|   3800|
| maria|   38|  Analista|   5500|
|marcos|   22|Estagiario|   1000|
| fabio|   35|Assistente|   2500|
|  luis|   40|   Gerente|  11700|
|   ana|   39|   Gerente|  15900|
+------+-----+----------+-------+



In [ ]:
df.show(5)

+------+-----+----------+-------+
|  nome|idade|     cargo|salario|
+------+-----+----------+-------+
|  joao|   25|  Analista|   3800|
| maria|   38|  Analista|   5500|
|marcos|   22|Estagiario|   1000|
| fabio|   35|Assistente|   2500|
|  luis|   40|   Gerente|  11700|
+------+-----+----------+-------+
only showing top 5 rows



In [ ]:
# Verificando colunas e tipos de dados das colunas:

df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- cargo: string (nullable = true)
 |-- salario: long (nullable = true)



In [ ]:
# filtrar elementos do DataFrame:

df.where("cargo == 'Gerente'").show()

+----+-----+-------+-------+
|nome|idade|  cargo|salario|
+----+-----+-------+-------+
|luis|   40|Gerente|  11700|
| ana|   39|Gerente|  15900|
+----+-----+-------+-------+



In [ ]:
# Filtre apenas o funcionarios que ganham menos de 3000 reais:

df.where("salario <= '3000'").show()

+------+-----+----------+-------+
|  nome|idade|     cargo|salario|
+------+-----+----------+-------+
|marcos|   22|Estagiario|   1000|
| fabio|   35|Assistente|   2500|
+------+-----+----------+-------+



In [ ]:
# segunda base de dados.

dados = [
    ("luciana", 22, "Assistente", 2700),
    ("carlos", 28, "Analista", 5500)
]

In [ ]:
df2 = spark.createDataFrame(dados, colunas)

In [ ]:
df2.show()

+-------+-----+----------+-------+
|   nome|idade|     cargo|salario|
+-------+-----+----------+-------+
|luciana|   22|Assistente|   2700|
| carlos|   28|  Analista|   5500|
+-------+-----+----------+-------+



In [ ]:
# Mesclando dois DataFrames:
# basicamente para isso uso o comando union e instancio nesse comando o df2

df = df.union(df2)

In [ ]:
df.show()

+-------+-----+----------+-------+
|   nome|idade|     cargo|salario|
+-------+-----+----------+-------+
|   joao|   25|  Analista|   3800|
|  maria|   38|  Analista|   5500|
| marcos|   22|Estagiario|   1000|
|  fabio|   35|Assistente|   2500|
|   luis|   40|   Gerente|  11700|
|    ana|   39|   Gerente|  15900|
|luciana|   22|Assistente|   2700|
| carlos|   28|  Analista|   5500|
+-------+-----+----------+-------+



In [ ]:
# Realizando algumas analises:

# Mostre a media salarial dos analistas dessa empresa:

df.where("cargo == 'Analista'").agg(
    f.mean("salario").alias("media_salario"),
    f.sum("salario").alias("soma_salario")
).show()

+-----------------+------------+
|    media_salario|soma_salario|
+-----------------+------------+
|4933.333333333333|       14800|
+-----------------+------------+



In [ ]:
# Importando arquivos para o Spark:
# "part*" no final do caminho da pasta do google drive, subir todos os arquivos que iniciem com part
url = "/content/drive/MyDrive/BasesSpark/empresas/part*"

# header=True significa que existe um cabeçalho nesse arquivo jque é a primeira linha.

empresas = spark.read.options(header=True).csv(
    url,
    sep=";", # Indentificar qual o parametro que separa as colunas.
    inferSchema=True # verificar a estrutra do arquivo e trás o arquivo com cada coluna com seu repectivo tipo, se não fizermos isso trará os dados tudo em string
)

In [ ]:
empresas.count()

4585679

In [ ]:
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|          0,00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|          0,00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|          0,00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|          0,00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|       4000,00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|          0,00|    5|           NULL|
| 8992|SHIROMA VEIC

In [ ]:
# Exibir quantidade de linhas especificas:

empresas.show(10, False)

+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj|razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306 |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355|BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820|REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARITIMOS|3

In [ ]:
# Solicitando que traga todas as informações, sem abreviações

empresas.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj |razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306  |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355 |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820 |REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARIT

In [ ]:
# Testando a alteração e visualizando.

empresas.withColumnsRenamed({
    "razao_social":"nome_empresa", # Colocar primeiro o nome anterior e depois o nome que deseja
    "qualificacao_responsavel" : "q_resp",
    "capital_social":"valor_investido"
}).show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|           0,00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|    49|           0,00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|    32|           0,00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|    50|           0,00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|    49|        4000,00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|           0,00|    5|           NULL|
| 8992|SHIROMA VEICULOS ...|             2062|    49|           0,00|    5|           NULL|
| 9091|CONTATOS BAR E LA...|             2062|    49|           0,00|    5|     

In [ ]:
# renomeando colunas

# IMPORTANTE: não podemos adicionar esse novo dataFrame, personalizado adicionando a função show.. pois se
# adicionarmos a função show.. ele irá adicionar um valor "none" na variavel empresas.. pois a função show
# não tem um return.. sendo assim ele não retorna nada na empresa.

empresas = empresas.withColumnsRenamed({
    "razao_social":"nome_empresa", # Colocar primeiro o nome anterior e depois o nome que deseja
    "qualificacao_responsavel" : "q_resp",
    "capital_social":"valor_investido"
})

In [ ]:
empresas.show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|           0,00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|    49|           0,00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|    32|           0,00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|    50|           0,00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|    49|        4000,00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|           0,00|    5|           NULL|
| 8992|SHIROMA VEICULOS ...|             2062|    49|           0,00|    5|           NULL|
| 9091|CONTATOS BAR E LA...|             2062|    49|           0,00|    5|     

In [ ]:
# Normalizando dados nulos:

# A função "na" faz esse tratamento de dados nulos.

# Se os dados forem string:
empresas = empresas.na.fill("N/A")

# se os dados forem numéricos:
empresas = empresas.na.fill(0)

In [ ]:
empresas.show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|           0,00|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|    49|           0,00|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|    32|           0,00|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|    50|           0,00|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|    49|        4000,00|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|           0,00|    5|            N/A|
| 8992|SHIROMA VEICULOS ...|             2062|    49|           0,00|    5|            N/A|
| 9091|CONTATOS BAR E LA...|             2062|    49|           0,00|    5|     

In [ ]:
# Traga empresas que possuem a palavra "RESTAURANTE" em sua descrição:

empresas.where("nome_empresa like '%RESTAURANTE%'")\
.orderBy("nome_empresa", ascending=False)\
.show(10, False)

+--------+---------------------------------------+-----------------+------+---------------+-----+---------------+
|cnpj    |nome_empresa                           |natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+--------+---------------------------------------+-----------------+------+---------------+-----+---------------+
|3295435 |ZUZUBEM BAR E RESTAURANTE LTDA.        |2062             |49    |0,00           |1    |N/A            |
|55979876|ZUPPO S RESTAURANTE LTDA               |2062             |49    |0,00           |5    |N/A            |
|4850826 |ZUPPA RESTAURANTE E BUFFET LTDA        |2062             |49    |0,00           |1    |N/A            |
|37679543|ZUMY RESTAURANTE LTDA                  |2062             |49    |50000,00       |1    |N/A            |
|30663090|ZUMPYARA BAR E RESTAURANTE LTDA        |2062             |49    |0,00           |5    |N/A            |
|36546265|ZULU 2000 BAR E RESTAURANTE LTDA       |2062             |49    |0,00         

In [ ]:
# para converter a coluna valor investido, devemos substituir a virgula por ponto.

# withColum = comando para fazer qualquer alteração nas colunas no dataframe

empresas.withColumn("valor_investido", f.regexp_replace("valor_investido", ",", ".")).show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|           0.00|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|    49|           0.00|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|    32|           0.00|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|    50|           0.00|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|    49|        4000.00|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|           0.00|    5|            N/A|
| 8992|SHIROMA VEICULOS ...|             2062|    49|           0.00|    5|            N/A|
| 9091|CONTATOS BAR E LA...|             2062|    49|           0.00|    5|     

In [ ]:
# Mostre a media de valor investido por natureza_juridica

empresas.groupBy("natureza_juridica").agg(
    f.mean("valor_investido").alias("media_investimento")
).show()

+-----------------+------------------+
|natureza_juridica|media_investimento|
+-----------------+------------------+
|             1139|              NULL|
|             3220|              NULL|
|             2259|              NULL|
|             1031|              NULL|
|             2313|              NULL|
|             2240|              NULL|
|             3069|              NULL|
|             1210|              NULL|
|             2062|              NULL|
|             1260|              NULL|
|             4014|              NULL|
|             2127|              NULL|
|             3239|              NULL|
|             4090|              NULL|
|             4120|              NULL|
|             1244|              NULL|
|             2038|              NULL|
|             2054|              NULL|
|             1155|              NULL|
|             1082|              NULL|
+-----------------+------------------+
only showing top 20 rows



In [ ]:
# efetivando a alteração

empresas = empresas.withColumn("valor_investido", f.regexp_replace("valor_investido", ",", "."))

In [ ]:
empresas.show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|           0.00|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|    49|           0.00|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|    32|           0.00|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|    50|           0.00|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|    49|        4000.00|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|           0.00|    5|            N/A|
| 8992|SHIROMA VEICULOS ...|             2062|    49|           0.00|    5|            N/A|
| 9091|CONTATOS BAR E LA...|             2062|    49|           0.00|    5|     

In [ ]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- nome_empresa: string (nullable = false)
 |-- natureza_juridica: integer (nullable = true)
 |-- q_resp: integer (nullable = true)
 |-- valor_investido: string (nullable = false)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = false)



In [ ]:
# Convertendo um campo string para float:

empresas.withColumn("valor_investido", empresas.valor_investido.cast("float")).show()

+-----+--------------------+-----------------+------+---------------+-----+---------------+
| cnpj|        nome_empresa|natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+-----+--------------------+-----------------+------+---------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|    49|            0.0|    1|            N/A|
| 1355|BRASILEIRO & OLIV...|             2062|    49|            0.0|    5|            N/A|
| 4820|REGISTRO DE IMOVE...|             3034|    32|            0.0|    5|            N/A|
| 5347|ROSELY APARECIDA ...|             2135|    50|            0.0|    5|            N/A|
| 6846|BADU E FILHOS TEC...|             2062|    49|         4000.0|    1|            N/A|
| 8416|  ELETRICA RUBI LTDA|             2062|    49|            0.0|    5|            N/A|
| 8992|SHIROMA VEICULOS ...|             2062|    49|            0.0|    5|            N/A|
| 9091|CONTATOS BAR E LA...|             2062|    49|            0.0|    5|     

In [ ]:
empresas = empresas.withColumn("valor_investido", empresas.valor_investido.cast("float"))

In [ ]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- nome_empresa: string (nullable = false)
 |-- natureza_juridica: integer (nullable = true)
 |-- q_resp: integer (nullable = true)
 |-- valor_investido: float (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = false)



In [ ]:
# fazendo um backup do dataframe

bkp = empresas

In [ ]:
# Mostrar a média de valor investido por natureza juridica:

empresas\
        .groupBy("natureza_juridica")\
        .agg(
            f.mean("valor_investido").alias("media_investimento")
        )\
        .orderBy("media_investimento", ascending=False)\
        .toPandas()

,natureza_juridica,media_investimento
0,2046,2.670461e+08
1,2038,6.751107e+07
2,2054,4.776993e+07
3,2011,3.142580e+07
4,2127,1.792158e+06
...,...,...
72,1309,0.000000e+00
73,1171,0.000000e+00
74,1317,0.000000e+00
75,1228,0.000000e+00


In [ ]:
# Criando um novo DataFrame somente com Hospitais:

hospitais = empresas\
                    .where("nome_empresa like 'HOSPITAL %'")

In [ ]:
hospitais.count()

366

In [ ]:
hospitais.show(10, False)

+--------+-----------------------------------------------------+-----------------+------+---------------+-----+---------------+
|cnpj    |nome_empresa                                         |natureza_juridica|q_resp|valor_investido|porte|ente_federativo|
+--------+-----------------------------------------------------+-----------------+------+---------------+-----+---------------+
|8992012 |HOSPITAL DE CLINICAS E MATERNIDADE DA ZONA OESTE LTDA|2240             |49    |50000.0        |1    |N/A            |
|24970428|HOSPITAL MUNICIPAL NOSSA SENHORA APARECIDA           |1031             |16    |0.0            |5    |SERRANIA - MG  |
|35359145|HOSPITAL SAO FRANCISCO DE BARREIRAS LTDA             |2062             |5     |2.6667E7       |5    |N/A            |
|37437944|HOSPITAL NOVA BRASILIA LTDA                          |2062             |49    |10000.0        |1    |N/A            |
|36408815|HOSPITAL VETERINARIO MUNDO ANIMAL LTDA               |2062             |49    |20000.0        

In [ ]:
"""
Atividade em sala de aula:

1 - Mude o nome da coluna "natureza_juridica" para "natureza" e da coluna "ente_federativo" para "ente".

2 - Verifique quantas empresas estão em recuperação judicial (essa informação está no nome da empresa,
de forma padronizada).

3 - Crie um novo DataFrame somente com as empresas que possuem valor_investido > 500.000 e <= 1.000.000.
Quantas empresas estão nessa regra?
"""

'\nAtividade em sala de aula:\n\n1 - Mude o nome da coluna "natureza_juridica" para "natureza" e da coluna "ente_federativo" para "ente".\n\n2 - Verifique quantas empresas estão em recuperação judicial (essa informação está no nome da empresa, \nde forma padronizada).\n\n3 - Crie um novo DataFrame somente com as empresas que possuem valor_investido > 500.000 e <= 1.000.000. \nQuantas empresas estão nessa regra?\n'

In [ ]:
# 1 - Mude o nome da coluna "natureza_juridica" para "natureza" e da coluna "ente_federativo" para "ente".

empresas.withColumnsRenamed({
    "natureza_juridica":"natureza",
    "ente_federativo":"ente"
}).show()


+-----+--------------------+--------+------+---------------+-----+----+
| cnpj|        nome_empresa|natureza|q_resp|valor_investido|porte|ente|
+-----+--------------------+--------+------+---------------+-----+----+
|  306|FRANCAMAR REFRIGE...|    2240|    49|            0.0|    1| N/A|
| 1355|BRASILEIRO & OLIV...|    2062|    49|            0.0|    5| N/A|
| 4820|REGISTRO DE IMOVE...|    3034|    32|            0.0|    5| N/A|
| 5347|ROSELY APARECIDA ...|    2135|    50|            0.0|    5| N/A|
| 6846|BADU E FILHOS TEC...|    2062|    49|         4000.0|    1| N/A|
| 8416|  ELETRICA RUBI LTDA|    2062|    49|            0.0|    5| N/A|
| 8992|SHIROMA VEICULOS ...|    2062|    49|            0.0|    5| N/A|
| 9091|CONTATOS BAR E LA...|    2062|    49|            0.0|    5| N/A|
| 9614|ANTONIA APARECIDA...|    2135|    50|            0.0|    5| N/A|
| 9896|DORACY CORAT DA C...|    2135|    50|            0.0|    5| N/A|
|12112|LANCHONETE RIO VE...|    2062|    49|            0.0|    

In [ ]:
# 2 - Verifique quantas empresas estão em recuperação judicial (essa informação está no nome da empresa,
# de forma padronizada).


recuperacao_judicial = empresas.where("nome_empresa like '%RECUPERACAO%'")
recuperacao_judicial.count()

939

In [ ]:
# 3 - Crie um novo DataFrame somente com as empresas que possuem valor_investido > 500.000 e <= 1.000.000.
# Quantas empresas estão nessa regra?

valor_investido = empresas.where("valor_investido > 500000 and valor_investido < 1000000")
valor_investido.count()

10378